In [23]:
from sklearn.datasets import load_iris
from math import ceil
import numpy as np

In [2]:
df = load_iris()

In [3]:
features = df['data'] 
target = df['target']

In [4]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.33, random_state = 83)

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()

features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

In [8]:
from keras.utils import to_categorical

In [9]:
target_train = to_categorical(target_train)
target_test = to_categorical(target_test)

In [11]:
import torch
import torch.nn as nn
from torch.optim import Adam

In [31]:
features_train = torch.FloatTensor(features_train)
features_test = torch.FloatTensor(features_test)
target_train = torch.FloatTensor(target_train)
target_test = torch.FloatTensor(target_test)

In [32]:
n_in_neurons = 4
n_hidden_neurons_1 = 30
n_hidden_neurons_2 = 20
n_out_neurons = 3

In [33]:
class Model(nn.Module):
    def __init__(self,n_in_neurons, n_hidden_neurons_1, 
                 n_hidden_neurons_2, n_out_neurons):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.ReLU()
        self.dp2 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.bn2 = nn.BatchNorm1d(n_hidden_neurons_2)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)
        self.act3 = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.dp2(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.fc3(x)
        x = self.act3(x)
        
        return x

In [34]:
net = Model(n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, 
            n_out_neurons)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
num_epochs = 100
batch_size = 5
num_bathces = ceil(len(features_train)/batch_size)

In [35]:
for epoch in range(num_epochs):
    order = np.random.permutation(len(features_train))
    
    for batch_idx in range(num_bathces):
        srart_index = batch_idx*batch_size
        optimizer.zero_grad()
        
        batch_indexes = order[srart_index:(srart_index+batch_size)]
        X_batch = features_train[batch_indexes]
        y_batch = target_train[batch_indexes]
        
        preds = net.forward(X_batch).flatten()
        loss_value = loss(preds, y_batch.flatten())
        loss_value.backward()
        optimizer.step()
        
    if epoch % 10 == 0 or epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(features_test)
        print('Категориальная кросс энтропия = :', 
              loss(target_test, test_preds))

Категориальная кросс энтропия = : tensor(1.1833, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.7039, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.6087, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5994, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.6000, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5971, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5975, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5951, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5945, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5920, grad_fn=<DivBackward1>)
Категориальная кросс энтропия = : tensor(0.5956, grad_fn=<DivBackward1>)


In [36]:
net.eval()
test_preds = net.forward(features_test)

In [37]:
test_preds

tensor([[8.9685e-12, 2.1259e-07, 1.0000e+00],
        [9.9895e-01, 1.0487e-03, 1.1240e-15],
        [5.4513e-08, 1.1911e-04, 9.9988e-01],
        [9.9938e-01, 6.2495e-04, 2.1103e-14],
        [1.6532e-08, 2.2926e-05, 9.9998e-01],
        [4.0395e-05, 9.0008e-01, 9.9875e-02],
        [9.9992e-01, 8.4452e-05, 3.0252e-15],
        [9.9992e-01, 7.6844e-05, 7.1767e-16],
        [9.9986e-01, 1.4219e-04, 6.2625e-15],
        [9.4060e-10, 2.0757e-06, 1.0000e+00],
        [9.9994e-01, 5.9178e-05, 8.0684e-16],
        [1.6545e-04, 9.9970e-01, 1.3303e-04],
        [3.9891e-06, 3.1388e-02, 9.6861e-01],
        [1.4542e-09, 4.1828e-06, 1.0000e+00],
        [5.2813e-07, 1.6252e-03, 9.9837e-01],
        [4.8243e-05, 9.9919e-01, 7.6404e-04],
        [9.9986e-01, 1.4422e-04, 2.2523e-14],
        [5.6356e-05, 9.9994e-01, 7.5484e-06],
        [2.7285e-05, 9.9990e-01, 6.7885e-05],
        [1.9141e-05, 9.9998e-01, 3.2547e-06],
        [1.9483e-08, 2.5305e-05, 9.9997e-01],
        [6.5556e-05, 9.9993e-01, 6